<div style=" background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px 0; color:#fff; text-align: center"> Timestamp conversion
</h2>
</div>

In [5]:
import biblio_modules

import datetime
from dateutil import parser
import time
import pytz

import pyarrow as py
import pandas as pd
import numpy as np

In [6]:
transactions_processing = pd.read_csv("../Source/Transactions.csv", sep=";")
transactions_processing

,id_prod,date,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


<div style="background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px; color:#fff; "> Traitement Timestamp  </h2>
<h3 style="margin: auto; padding: 10px; color:#fff; "> --> Début du data processing  </h3>
</div>

In [4]:
#Je coupe la colonne date et time en deux colonnes
transactions_processing[['Date','Time']] = transactions_processing['date'].str.split(expand=True)
#Je converti les date en DateTime
transactions_processing['Date'] = pd.to_datetime(transactions_processing['Date'])

In [5]:
#J'ajoute 1 jour au date sur lesquelles l'heures est de 24
transactions_processing.loc[transactions_processing['Time'].str.startswith('24'), 'Date'] = transactions_processing.loc[transactions_processing['Time'].str.startswith('24'), 'Date'] + pd.DateOffset(days = 1)


In [6]:
#Je change les time '24' en '00' pour uniformiser les times
condition = transactions_processing['Time'].str.startswith('24')
transactions_processing.loc[condition, 'Time'] = '00' + transactions_processing.loc[condition, 'Time'].str[2:]
transactions_processing[transactions_processing['date'].str.contains(' 24:')]


,id_prod,date,session_id,client_id,Date,Time
198325,1_400,2021-10-02 24:04:58.422464,s_107311,c_2513,2021-10-03,00:04:58.422464
198326,1_601,2021-10-02 24:04:59.254020,s_107311,c_2513,2021-10-03,00:04:59.254020
198327,1_735,2021-10-02 24:04:59.422464,s_107311,c_2513,2021-10-03,00:04:59.422464
198328,1_351,2021-10-02 24:04:59.913108,s_107311,c_2513,2021-10-03,00:04:59.913108
198329,1_400,2021-10-02 24:06:02.588166,s_107205,c_5036,2021-10-03,00:06:02.588166
...,...,...,...,...,...,...
221108,1_445,2021-10-26 24:36:01.240212,s_105041,c_3949,2021-10-27,00:36:01.240212
221109,1_296,2021-10-26 24:36:21.240212,s_105041,c_3949,2021-10-27,00:36:21.240212
221820,1_45,2021-10-27 24:18:21.505552,s_98882,c_7656,2021-10-28,00:18:21.505552
221821,1_431,2021-10-27 24:20:46.744883,s_99594,c_8073,2021-10-28,00:20:46.744883


In [7]:
#Je reconvertis mes date en string
transactions_processing['Date'] = transactions_processing['Date'].astype(str)
transactions_processing.head()

,id_prod,date,session_id,client_id,Date,Time
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,2021-03-01,00:01:07.843138
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,2021-03-01,00:02:26.047414
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,2021-03-01,00:02:38.311413
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,2021-03-01,00:04:54.559692
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,2021-03-01,00:05:18.801198


In [8]:
#Je sépare mes colonnes date et time en colonne distincte
transactions_processing[['Year','Month','Day']] = transactions_processing['Date'].str.split(pat="-", expand=True)
transactions_processing[['Hour','Minute','Second']] = transactions_processing['Time'].str.split(pat=":", expand=True)
transactions_processing[['Second','Ns']] = transactions_processing['Second'].str.split(pat=".", expand=True)
transactions_processing.drop(columns=['date', 'Date', 'Time'], inplace=True)
transactions_processing

,id_prod,session_id,client_id,Year,Month,Day,Hour,Minute,Second,Ns
0,0_1259,s_1,c_329,2021,03,01,00,01,07,843138
1,0_1390,s_2,c_664,2021,03,01,00,02,26,047414
2,0_1352,s_3,c_580,2021,03,01,00,02,38,311413
3,0_1458,s_4,c_7912,2021,03,01,00,04,54,559692
4,0_1358,s_5,c_2033,2021,03,01,00,05,18,801198
...,...,...,...,...,...,...,...,...,...,...
687529,1_508,s_348444,c_3573,2023,02,28,23,49,03,148402
687530,2_37,s_348445,c_50,2023,02,28,23,51,29,318531
687531,1_695,s_348446,c_488,2023,02,28,23,53,18,929676
687532,0_1547,s_348447,c_4848,2023,02,28,23,58,00,107815


In [10]:
#Je convertis les colonne date et time en integer
transactions_processing[['Year', 'Month', 'Day', 'Hour','Minute', 'Second', 'Ns']] = transactions_processing[['Year', 'Month', 'Day', 'Hour','Minute', 'Second', 'Ns']].astype(int)

In [11]:
#je fusionne toute mes colonnes en timestamps
transactions_processing['Timestamp'] = transactions_processing.apply(
    lambda x: pd.Timestamp(year=x.Year,month=x.Month,day=x.Day,hour=x.Hour,minute=x.Minute,second=x.Second,microsecond=x.Ns), axis=1)
#Je drop les colonnes individuels
transactions_processing.drop(columns=['Year', 'Month', 'Day', 'Hour','Minute', 'Second', 'Ns'], inplace=True)

In [13]:
transactions_processing.head()

,id_prod,session_id,client_id,Timestamp
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414
2,0_1352,s_3,c_580,2021-03-01 00:02:38.311413
3,0_1458,s_4,c_7912,2021-03-01 00:04:54.559692
4,0_1358,s_5,c_2033,2021-03-01 00:05:18.801198


In [14]:
transactions_processing.index.dtype

dtype('int64')

In [16]:
#Je reorganise mes colonnes
transactions_processing = transactions_processing[['id_prod', 'Timestamp', 'session_id', 'client_id']]

In [17]:
transactions_processing

,id_prod,Timestamp,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


In [18]:
#je dois donc réculer d'1h tout les enregistrement entre le 28 mars 2021 à 2h du matin et le 31 octobre 2021 à 3h du matin

In [36]:
x= transactions_processing.loc[
    (transactions_processing['Timestamp'] > datetime.datetime(year=2021, month=10, day=31, hour=2 )) 
    & 
    (transactions_processing['Timestamp'] < datetime.datetime(year=2021, month=10, day=31, hour=3))]
x

,id_prod,Timestamp,session_id,client_id
224942,1_202,2021-10-31 02:00:06.704413,s_112515,c_7342
224943,0_1611,2021-10-31 02:01:09.803700,s_112516,c_5756
224944,0_1525,2021-10-31 02:01:47.590195,s_112515,c_7342
224945,1_451,2021-10-31 02:03:02.978253,s_112517,c_8112
224946,0_2074,2021-10-31 02:03:18.728733,s_112515,c_7342
...,...,...,...,...
225011,1_631,2021-10-31 02:53:41.989579,s_112542,c_5041
225012,0_2052,2021-10-31 02:55:21.051240,s_112543,c_3507
225013,1_381,2021-10-31 02:56:36.805619,s_112535,c_161
225014,1_312,2021-10-31 02:57:48.279426,s_112543,c_3507


In [ ]:
transactions_processing.loc[
    (transactions_processing['Timestamp'] > datetime.datetime(year=2021, month=3, day=28, hour=2 )) 
    & 
    (transactions_processing['Timestamp'] < datetime.datetime(year=2021, month=10, day=31, hour=4))]

In [16]:
transactions_processing.to_csv("../Source/Data_processed/timestamps_processed.csv", sep=";", index=False)

<div style="background-color: rgb(43, 197, 37);" >
<h2 style="margin: auto; padding: 20px; color:#fff; "> Traitement Timestamp  </h2>
<h3 style="margin: auto; padding: 10px; color:#fff; "> --> Fin du data processing  </h3>
</div>